In [56]:
import os
from openai import OpenAI
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever

os.environ["OPENAI_API_KEY"] = ""
job_id = 'ftjob-hjLV1XIM6ccPjzx6zTpAYw5z'

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
finetuned_job = client.fine_tuning.jobs.retrieve(job_id)
finetuned_model = finetuned_job.fine_tuned_model

llm_model = ChatOpenAI(model_name=finetuned_model, temperature=0.4)

Basic LLM Chain with system prompt

In [5]:
prompt = ChatPromptTemplate.from_template("""
                                          Your name is  Assistant. You are a personal assistant .
                                          Answer the user's question:
                                          Question: {question}""")

chain = prompt | llm_model
response = chain.invoke({"question": "What is  x?"})

Include Context in prompt using documents

In [29]:

def get_document():
    loader = PyPDFLoader("../_RAG_3.pdf")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs

def create_db(docs):
    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(docs, embedding = embeddings)
    return db

In [39]:
def create_chain(vectore_store):
    llm = ChatOpenAI(model_name=finetuned_model, temperature=0.4)
    
    prompt_context = ChatPromptTemplate.from_template("""
                                          Your name is  Assistant. You are a personal assistant .
                                          Answer the user's question:
                                          Context: {context}
                                          Question: {input}""")
    
    chain_context = create_stuff_documents_chain(llm = llm, prompt = prompt_context,)
    
    retriever = vectore_store.as_retriever(search_kwargs={"k": 2})
    retriever_chain = create_retrieval_chain(retriever, chain_context)
    
    return retriever_chain

In [35]:
vector_store = create_db(get_document())
main_chain = create_chain(vector_store)

response = main_chain.invoke({
    "input" : "what is the most expensive product of ?",
})

print(response["answer"])

The most expensive product of  is the Custom Series, which includes the CX1, CX2, C1, and C2 models.


Include chat history

In [59]:
def history_and_retriever(vector_store):
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Your name is  Assistant. You are a personal assistant . Answer the user's question based on the context: {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}")
    ])

    chain = create_stuff_documents_chain(llm = llm_model, prompt = prompt)
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})

    #combine chat history and latest user input
    retriever_prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        ("human", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
    ])

    # Generate history aware retriever
    history_aware_retriever = create_history_aware_retriever(llm=llm_model, retriever = retriever, prompt= retriever_prompt)
    
    retrieval_chain = create_retrieval_chain(history_aware_retriever,chain)
    
    return retrieval_chain



In [61]:
def prepare_conversation(main_chain,question,chat_history):
    response = main_chain.invoke({
    "input" : question,
    "chat_history" : chat_history
})
    return response["answer"]

In [1]:
if __name__== '__main__':
    vector_store = create_db(get_document())
    #main_chain = create_chain(vector_store)
    history_aware_chain = history_and_retriever(vector_store)
    
    chat_history = []
    
    while True:
        user_input = input("User: ")
        response = prepare_conversation(history_aware_chain, user_input,chat_history)
        #append user input to chat history
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=response))
        print("Human:",user_input )
        print("Assistant:",response)

NameError: name 'create_db' is not defined